In [1]:
import pandas as pd

df = pd.read_csv('file/public_result.csv')
df.head()

,question,answer,category02
0,여기 서명란에 펜이나 연필로 사인해도 되는 거 아니에요?,확인 결과 인감 외 사인도 동일 서명으로 처리됩니다.,17.민원 신청서 용어 질문
1,이런 전기 요금 할인처럼 비슷한 서비스 같은 것이 또 있나요?,네 소득기준과 세대원 특정 기준에 모두 충족하는 세대에게 에너지 바우처를 지원해 줍니다.,18.관내 복지 지원금 정보 질문
2,여기 소포 보낼 때 한 번에 결제하고 그런 서비스는 지원 안 하나요?,죄송합니다. 선불로 처리될 수밖에 없는 점 양해 부탁드립니다.,18.관내 복지 지원금 정보 질문
3,이런데 로봇이 있어? 신기하네 사업 지원금 관련 담당자는 어디 계실까요?,담당자분 성함 알려주시면 확인 후 안내 도와드리겠습니다.,13.입장 및 민원서비스 이용 안내
4,서류 가지고 왔어요. 여기에 제출하면 되죠?,성적장학금의 경우 재학 증명서와 성적 증명서가 필요합니다.,15.준비서류 확인


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel
import os

# BERT 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")
model = AutoModel.from_pretrained("beomi/KcBERT-base")

# 질문 리스트
questions = df['question'].tolist()
answers = df['answer'].tolist()
# # 질문들의 임베딩 벡터 미리 계산
# def compute_question_embeddings(questions):
#     embeddings = []
#     for question in questions:
#         inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True)
#         with torch.no_grad():
#             outputs = model(**inputs)
#         # 문장 전체의 임베딩을 평균으로 취함
#         embedding = outputs.last_hidden_state.mean(dim=1)
#         embeddings.append(embedding)
#     return embeddings

# # 질문에 대한 임베딩 계산
# question_embeddings = compute_question_embeddings(questions)

# # 저장할 폴더 경로
# save_dir = 'embeddings'

# # 폴더가 없으면 생성
# os.makedirs(save_dir, exist_ok=True)

# # 저장할 파일 경로
# save_path = os.path.join(save_dir, 'question_embeddings.pth')

# # 임베딩 저장
# torch.save(question_embeddings, save_path)


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch.nn.functional as F
import torch

question_embeddings = torch.load('embeddings/public_question_embeddings.pth', weights_only=True)

# 입력 문장을 임베딩으로 변환
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # 임베딩 벡터 반환

# 입력 질문과 사전 임베딩 간의 코사인 유사도 계산
def find_most_similar_answer(input_question, question_embeddings, answers):
    input_embedding = get_embedding(input_question)  # 입력 질문 임베딩

    max_similarity = -1
    best_answer = None

    # 각 사전 계산된 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        similarity = F.cosine_similarity(input_embedding, question_embedding).item()

        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity

# 입력 질문 예시
input_question = "서류제출은 어디에 해요?"

# 유사도 기반 답변 찾기
answer, max_similarity = find_most_similar_answer(input_question, question_embeddings, answers)
print("챗봇 답변:", answer)
print("유사도:", max_similarity)


챗봇 답변: 서류 제출 창구 번호를 확인 중입니다. 잠시만 기다려 주십시오.
유사도: 0.8327068090438843


In [5]:
# 입력 질문 예시
input_question = "서류 가지고 왔어요. 여기에 제출하면 되죠?"

# 유사도 기반 답변 찾기
answer, max_similarity = find_most_similar_answer(input_question, question_embeddings, answers)
print("챗봇 답변:", answer)
print("유사도:", max_similarity)

챗봇 답변: 성적장학금의 경우 재학 증명서와 성적 증명서가 필요합니다.
유사도: 1.0000001192092896


In [6]:
df['category02'].unique()

array(['17.민원 신청서 용어 질문', '18.관내 복지 지원금 정보 질문', '13.입장 및 민원서비스 이용 안내',
       '15.준비서류 확인', '14.민원별 번호표 발급 및 창구 안내', '16.민원 신청서 작성 안내',
       '16.민원 신청서 작성 안내내'], dtype=object)

In [7]:
df['category02'].value_counts()

category02
16.민원 신청서 작성 안내          8509
13.입장 및 민원서비스 이용 안내      8417
15.준비서류 확인               8415
18.관내 복지 지원금 정보 질문       8406
14.민원별 번호표 발급 및 창구 안내    8393
17.민원 신청서 용어 질문          8386
16.민원 신청서 작성 안내내            1
Name: count, dtype: int64

In [8]:
df[df['category02'] == '16.민원 신청서 작성 안내내']

,question,answer,category02
4435,양식 안에 있는 접수 보건소명이랑 접수일자도 제가 작성해야 되나요?,신청서 제일 상단에 담당 공무원이 기재하는 사항은 공란으로 비워두셔도 됩니다.,16.민원 신청서 작성 안내내


In [4]:
# 정규표현식을 사용하여 카테고리 앞에 있는 숫자와 점을 제거
df['category02'] = df['category02'].str.replace(r'^\d+\.', '', regex=True)

# '진료 접수 안내'와 '진료 접수'를 같은 카테고리로 변경
df['category02'] = df['category02'].replace({'민원 신청서 작성 안내내': '민원 신청서 작성 안내'})

# 고유한 카테고리 값 추출
unique_categories = df['category02'].unique()
print(unique_categories)

['민원 신청서 용어 질문' '관내 복지 지원금 정보 질문' '입장 및 민원서비스 이용 안내' '준비서류 확인'
 '민원별 번호표 발급 및 창구 안내' '민원 신청서 작성 안내']


In [5]:
# 카테고리와 고유 번호 매핑
category_to_label = {category: idx for idx, category in enumerate(unique_categories)}

# 레이블 추가
df['label'] = df['category02'].map(category_to_label)

# 결과 확인
df.head()
df['label'].unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [6]:
df.to_csv('public_result_label.csv', index=False)

In [6]:
from sklearn.model_selection import train_test_split
from collections import Counter
# import random
# CUDA 디버깅을 위한 환경 변수 설정
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# 질문과 답변을 결합
df['input'] = df['question'] + " [SEP] " + df['answer']  # [SEP]는 BERT에서 문장을 구분하는 토큰입니다.

# 훈련 데이터와 평가 데이터로 분리
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    df['input'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

# 결과 확인
print("Updated train labels:", Counter(train_labels))
print(set(train_labels))
print(set(eval_labels))

Updated train labels: Counter({5: 6791, 1: 6754, 2: 6754, 0: 6727, 4: 6712, 3: 6683})
{0, 1, 2, 3, 4, 5}
{0, 1, 2, 3, 4, 5}


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")

# 토크나이징
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
eval_encodings = tokenizer(eval_texts, truncation=True, padding=True)


In [8]:
import torch


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, num_classes):
        self.encodings = encodings
        self.labels = labels

        # 레이블 유효성 검사
        assert all(label >= 0 for label in self.labels), "Negative label found!"
        assert all(label < num_classes for label in self.labels), "Label out of range!"

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# 데이터셋 생성
num_classes = 6  # 모델의 클래스 수
train_dataset = CustomDataset(train_encodings, train_labels, num_classes)
eval_dataset = CustomDataset(eval_encodings, eval_labels, num_classes)


In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import DataLoader

model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base", num_labels=num_classes)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    eval_strategy="epoch",
    logging_dir='./logs',
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# 모델 훈련
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/7581 [00:00<?, ?it/s]c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  7%|▋         | 500/7581 [01:31<23:36,  5.00it/s]

{'loss': 0.5114, 'grad_norm': 22.280275344848633, 'learning_rate': 4.6702282020841584e-05, 'epoch': 0.2}


 13%|█▎        | 1000/7581 [03:53<26:23,  4.16it/s]  

{'loss': 0.213, 'grad_norm': 5.31935977935791, 'learning_rate': 4.340456404168316e-05, 'epoch': 0.4}


 20%|█▉        | 1500/7581 [06:27<29:18,  3.46it/s]   

...

 99%|█████████▉| 7500/7581 [41:35<00:26,  3.07it/s]  

{'loss': 0.0045, 'grad_norm': 0.022359976544976234, 'learning_rate': 5.342303126236644e-07, 'epoch': 2.97}


                                                   
100%|██████████| 7581/7581 [43:45<00:00,  2.89it/s]

{'eval_loss': 0.009955658577382565, 'eval_runtime': 42.9413, 'eval_samples_per_second': 235.345, 'eval_steps_per_second': 29.436, 'epoch': 3.0}
{'train_runtime': 2625.2796, 'train_samples_per_second': 46.191, 'train_steps_per_second': 2.888, 'train_loss': 0.07505480861965734, 'epoch': 3.0}


TrainOutput(global_step=7581, training_loss=0.07505480861965734, metrics={'train_runtime': 2625.2796, 'train_samples_per_second': 46.191, 'train_steps_per_second': 2.888, 'total_flos': 4798480853248236.0, 'train_loss': 0.07505480861965734, 'epoch': 3.0})

In [16]:
trainer.evaluate()

100%|██████████| 1264/1264 [00:41<00:00, 30.13it/s]


{'eval_loss': 0.009955658577382565,
 'eval_runtime': 41.695,
 'eval_samples_per_second': 242.379,
 'eval_steps_per_second': 30.315,
 'epoch': 3.0}

In [17]:
import os
import torch

# 저장할 폴더 경로 설정
save_dir = 'model'

# 폴더가 없으면 생성
os.makedirs(save_dir, exist_ok=True)

# 저장할 파일 경로 설정
save_path = os.path.join(save_dir, 'public_model_weights.pth')

# 모델의 상태 사전(state_dict) 저장 -> 호환성이 더 좋음
torch.save(model.state_dict(), save_path)

In [9]:
import torch
from transformers import AutoModelForSequenceClassification

# 모델 구조를 정의한 코드
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base", num_labels=num_classes)

# 모델 가중치를  로드
model.load_state_dict(torch.load('model/public_model_weights.pth', weights_only=True))

# 평가 모드로 전환
model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [20]:
import os
import numpy as np
import torch

questions = df['question'].tolist()
answers = df['answer'].tolist()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

def get_embeddings(questions, tokenizer, model):
    model.eval()  # 평가 모드로 전환
    embeddings = []

    with torch.no_grad():
        for question in questions:
            # 입력 토큰화 (입력을 GPU로 이동)
            inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True).to(device)

            # 출력 얻기
            outputs = model(**inputs, output_hidden_states=True)  # hidden_states를 반환하도록 설정
            
            # BERT의 마지막 레이어 hidden_states에서 [CLS] 토큰의 임베딩을 사용
            cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [batch_size, hidden_size]

            # CPU로 이동하고 numpy 배열로 변환하여 임베딩 목록에 추가
            embeddings.append(cls_embedding.squeeze().cpu().numpy())

    return np.array(embeddings)  # 모든 임베딩을 NumPy 배열로 반환


# training 후 새로운 임베딩 생성
question_embeddings = get_embeddings(questions, tokenizer, model)

# 저장할 폴더 경로
save_dir = 'embeddings'

# 폴더가 없으면 생성
os.makedirs(save_dir, exist_ok=True)

# 저장할 파일 경로
save_path = os.path.join(save_dir, 'public_train_question_embeddings.pth')

# 임베딩 저장
torch.save(question_embeddings, save_path)

In [22]:
import torch
import torch.nn.functional as F
import numpy as np

# numpy의 _reconstruct 함수를 허용 목록에 추가
torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])

# 데이터프레임과 디바이스 설정
device = torch.device('cpu')

# 신뢰할 수 있는 데이터 파일을 로드하고 GPU로 이동
question_embeddings = torch.load('embeddings/public_train_question_embeddings.pth')
question_embeddings = [torch.tensor(embedding).to(device) for embedding in question_embeddings]  # 모든 임베딩을 GPU로 이동

# 평가 모드로 설정
model.eval()


def get_embedding(input_question, tokenizer, model, device):
    # 입력 문장을 토크나이즈하고 GPU/CPU로 이동
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        # hidden states를 포함하도록 설정
        outputs = model(**inputs, output_hidden_states=True)
        
        # 마지막 hidden state에서 [CLS] 토큰의 임베딩을 가져옴
        cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [CLS] 토큰의 임베딩
    
    return cls_embedding.squeeze().to(device)  # 텐서를 GPU로 반환

# 코사인 유사도를 계산하여 가장 유사한 답변을 찾는 함수
def find_most_similar_answer_cosine(input_question, question_embeddings, answers, tokenizer, model, device):
    # 입력 질문 임베딩 생성
    input_embedding = get_embedding(input_question, tokenizer, model, device)

    max_similarity = -1
    best_answer = None
    
    # 각 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        # 코사인 유사도 계산
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), question_embedding.unsqueeze(0)).item()
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity

# 챗봇 응답 함수
def chatbot_response(input_question, tokenizer, model, question_embeddings, answers, df, device):
    # 1차 필터링: 분류 모델로 레이블 예측
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=-1).item()

    # 2차 필터링: 같은 카테고리 내에서 코사인 유사도 계산
    # 같은 레이블의 질문들과 임베딩 필터링
    filtered_df = df[df['label'] == predicted_label]
    filtered_indices = filtered_df.index.tolist()

    # 필터링된 질문에 해당하는 미리 계산된 임베딩과 답변 가져오기
    filtered_question_embeddings = [question_embeddings[i] for i in filtered_indices]
    filtered_answers = [answers[i] for i in filtered_indices]

    # 코사인 유사도를 통해 가장 유사한 답변 찾기
    best_answer, cosine_similarity = find_most_similar_answer_cosine(input_question, filtered_question_embeddings, filtered_answers, tokenizer, model, device)
    
    return best_answer, cosine_similarity, predicted_label


c:\Temp\ipykernel_19416\594036569.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  question_embeddings = torch.load('embeddings/public_train_question_embeddings.pth')


In [23]:
# 예시 질문
input_question = "서류 제출은 어디에 해요?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(
    input_question, tokenizer, model, question_embeddings, answers, df, device
)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)

예측된 레이블: 4
최고 유사도 답변: 말씀하신 질문에 대한 답변으로, 상담 문의는 능력 개발 운영부입니다.
코사인 유사도: 0.8322391510009766


In [24]:
# 예시 질문
input_question = "서류 가지고 왔어요. 여기에 제출하면 되죠?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(
    input_question, tokenizer, model, question_embeddings, answers, df, device
)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)

예측된 레이블: 3
최고 유사도 답변: 성적장학금의 경우 재학 증명서와 성적 증명서가 필요합니다.
코사인 유사도: 1.0


In [10]:
import optuna
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from transformers import EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base", num_labels=num_classes)

# 성능 평가 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Accuracy
    accuracy = accuracy_score(labels, predictions)

    # F1 Score (macro average for multi-class classification)
    f1 = f1_score(labels, predictions, average="macro")

    return {
        "accuracy": accuracy,
        "f1": f1,
    }

# Optuna 최적화 목적 함수 정의
def objective(trial):
    # 하이퍼파라미터 제안
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True)
    batch_size = trial.suggest_int("batch_size", 8, 32)
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 5)

    # TrainingArguments 설정
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        learning_rate=learning_rate,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./logs',
        logging_steps=10,
        save_total_limit=1,
        load_best_model_at_end=True,
        evaluation_strategy="epoch",  # 평가 전략을 에포크 단위로 설정
    )

    # Trainer 설정
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,  # 성능 평가 함수 추가
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # early stopping 추가
    )

    # 모델 학습
    trainer.train()

    # 평가 및 최적화 지표 반환
    eval_result = trainer.evaluate()
    return eval_result["eval_loss"]

# Optuna 최적화 실행
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2024-10-13 20:50:56,328] A new study created in memory with name: no-name-7c268cd4-bcc6-4bf3-9b40-d921b6c38e23
c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/3792 [00:00<?, ?it/s]c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transfor

{'loss': 1.795, 'grad_norm': 14.408425331115723, 'learning_rate': 1.8705668652985585e-05, 'epoch': 0.01}


  1%|          | 20/3792 [00:06<17:35,  3.57it/s]

{'loss': 1.6787, 'grad_norm': 7.032289028167725, 'learning_rate': 1.8656208926245803e-05, 'epoch': 0.02}


  1%|          | 30/3792 [00:09<17:51,  3.51it/s]

...

100%|██████████| 2788/2788 [28:39<00:00,  1.62it/s]


{'train_runtime': 1719.7749, 'train_samples_per_second': 47.007, 'train_steps_per_second': 1.621, 'train_loss': 0.013250855881611966, 'epoch': 2.0}


 30%|███       | 383/1264 [00:14<00:36, 24.21it/s][W 2024-10-13 23:16:06,478] Trial 2 failed with parameters: {'learning_rate': 3.160271784760812e-05, 'batch_size': 29, 'num_train_epochs': 2} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "c:\Temp\ipykernel_18704\3077769053.py", line 61, in objective
    eval_result = trainer.evaluate()
  File "c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\trainer.py", line 3868, in evaluate
    output = eval_loop(
  File "c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\trainer.py", line 4061, in evaluation_loop
    losses, logits, labels = self.prediction_step(model, inputs, prediction_loss_only, ignore_keys=ignore_keys)
  File "c:\Users\user\AppData\Local\Programs\Pyth

KeyboardInterrupt: 

 30%|███       | 385/1264 [00:25<00:36, 24.21it/s]

In [39]:
# 최적화된 하이퍼파라미터 출력
print(f"Best hyperparameters: {study.best_trial.params}")

Best hyperparameters: {'learning_rate': 2.365573325346666e-05, 'batch_size': 13, 'num_train_epochs': 2}


In [40]:
eval_results = trainer.evaluate()
print(eval_results)


# {'eval_loss': 0.009955658577382565,
#  'eval_runtime': 41.695,
#  'eval_samples_per_second': 242.379,
#  'eval_steps_per_second': 30.315,
#  'epoch': 3.0}

100%|██████████| 1264/1264 [01:01<00:00, 20.48it/s]

{'eval_loss': 0.009955658577382565, 'eval_runtime': 61.4753, 'eval_samples_per_second': 164.391, 'eval_steps_per_second': 20.561, 'epoch': 3.0}


In [41]:
# Fine-tuning된 모델 저장
model.save_pretrained("./public_fine_tuned_model_previous")
tokenizer.save_pretrained("./public_fine_tuned_model_previous")

('./public_fine_tuned_model\\tokenizer_config.json',
 './public_fine_tuned_model\\special_tokens_map.json',
 './public_fine_tuned_model\\vocab.txt',
 './public_fine_tuned_model\\added_tokens.json',
 './public_fine_tuned_model\\tokenizer.json')

In [42]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# fine-tuning된 모델과 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained("./public_fine_tuned_model_previous")
tokenizer = AutoTokenizer.from_pretrained("./public_fine_tuned_model_previous")


In [43]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [44]:
questions = df['question'].tolist()
answers = df['answer'].tolist()

def get_embeddings(questions, tokenizer, model):
    model.eval()  # 평가 모드로 전환
    embeddings = []

    with torch.no_grad():
        for question in questions:
            # 입력 토큰화
            inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True)

            # 출력 얻기
            outputs = model(**inputs, output_hidden_states=True)  # hidden_states를 반환하도록 설정
            
            # BERT의 마지막 레이어 hidden_states에서 [CLS] 토큰의 임베딩을 사용
            cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [batch_size, hidden_size]
            
            # numpy 배열로 변환하여 임베딩 목록에 추가
            embeddings.append(cls_embedding.squeeze().numpy())

    return np.array(embeddings)  # 모든 임베딩을 NumPy 배열로 반환


# fine-tuning 후 새로운 임베딩 생성
question_embeddings = get_embeddings(questions, tokenizer, model)

# 저장할 폴더 경로
save_dir = 'embeddings'

# 폴더가 없으면 생성
os.makedirs(save_dir, exist_ok=True)

# 저장할 파일 경로
save_path = os.path.join(save_dir, 'public_previous_optimized_question_embeddings.pth')

# 임베딩 저장
torch.save(question_embeddings, save_path)

In [45]:
import torch
import torch.nn.functional as F
import numpy as np

# numpy의 _reconstruct 함수를 허용 목록에 추가
torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])

# 신뢰할 수 있는 데이터 파일을 로드
question_embeddings = torch.load('embeddings/public_previous_optimized_question_embeddings.pth')

# 평가 모드로 설정
model.eval()

def get_embedding(input_question, tokenizer, model):
    # 입력 문장을 토크나이즈
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True)
    
    with torch.no_grad():
        # hidden states를 포함하도록 설정
        outputs = model(**inputs, output_hidden_states=True)
        
        # 마지막 hidden state에서 [CLS] 토큰의 임베딩을 가져옴
        cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [CLS] 토큰의 임베딩
        
    return cls_embedding.squeeze().numpy()  # numpy 배열로 반환



# 코사인 유사도를 계산하여 가장 유사한 답변을 찾는 함수
def find_most_similar_answer_cosine(input_question, question_embeddings, answers, tokenizer, model):
    # 입력 질문 임베딩 생성
    input_embedding = get_embedding(input_question, tokenizer, model)

    max_similarity = -1
    best_answer = None
    
    # 각 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        # question_embedding을 텐서로 변환하고 차원 맞추기
        question_embedding_tensor = torch.tensor(question_embedding).unsqueeze(0)  # (1, 768)
        
        # input_embedding도 텐서로 변환하고 차원 맞추기
        input_embedding_tensor = torch.tensor(input_embedding).unsqueeze(0)  # (1, 768)
        
        # 코사인 유사도 계산
        similarity = F.cosine_similarity(input_embedding_tensor, question_embedding_tensor).item()
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity  # 유사도 반환 추가

# 챗봇 응답 함수
def chatbot_response(input_question, tokenizer, model, question_embeddings, answers):
    # 1차 필터링: 분류 모델로 레이블 예측
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=-1).item()

    # 2차 필터링: 같은 카테고리 내에서 코사인 유사도 계산
    # 같은 레이블의 질문들과 임베딩 필터링
    filtered_df = df[df['label'] == predicted_label]
    filtered_indices = filtered_df.index.tolist()

    # 필터링된 질문에 해당하는 미리 계산된 임베딩과 답변 가져오기
    filtered_question_embeddings = [question_embeddings[i] for i in filtered_indices]
    filtered_answers = [answers[i] for i in filtered_indices]

    # 코사인 유사도를 통해 가장 유사한 답변 찾기
    best_answer, cosine_similarity = find_most_similar_answer_cosine(input_question, filtered_question_embeddings, filtered_answers, tokenizer, model)
    
    return best_answer, cosine_similarity, predicted_label  # 세 가지 값 반환


c:\Temp\ipykernel_20856\650173660.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  question_embeddings = torch.load('embeddings/public_optimized_question_embeddings.pth')

In [46]:
# 예시 질문
input_question = "서류 제출은 어디에 해요?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(input_question, tokenizer, model, question_embeddings, answers)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)



예측된 레이블: 3
최고 유사도 답변: 부부 각자의 가족관계 증명서와 혼인관계 증명서, 등본, 협의이혼 의사 확인 신청서가 필요합니다.
코사인 유사도: 1.000000238418579


In [51]:
# 예시 질문
input_question = "서류 가지고 왔는데 여기에 제출하면 되죠?"
# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(input_question, tokenizer, model, question_embeddings, answers)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)

예측된 레이블: 3
최고 유사도 답변: 준비 서류 관련 질문을 접수했습니다. 동의서도 함께 필요합니다.
코사인 유사도: 1.0000001192092896


In [47]:
# 예시 질문
input_question = "대기가 너무 길잖아요."

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(input_question, tokenizer, model, question_embeddings, answers)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)

예측된 레이블: 4
최고 유사도 답변: 저를 따라오시면 안내해 드리겠습니다. 번호표를 먼저 발급받아주세요.
코사인 유사도: 1.0000001192092896
